In [ ]:
!python -m pip install --upgrade pip
!pip install gym
!pip install numpy
!pip install torch
!pip install torchvision
!pip install matplotlib
!pip install pygame
!pip install setuptools wheel
!pip install build
!pip install gym[classic_control]


In [4]:
import gym
import torch
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque

# Create the environment with new_step_api=True
env = gym.make("CartPole-v1", new_step_api=True)

# Hyperparameters
GAMMA = 0.99
LR = 0.001
EPSILON = 1.0
EPSILON_MIN = 0.01
EPSILON_DECAY = 0.995
MEMORY_SIZE = 10000
BATCH_SIZE = 64
TARGET_UPDATE = 10

class DQN(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# Create networks
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
policy_net = DQN(state_dim, action_dim)
target_net = DQN(state_dim, action_dim)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.Adam(policy_net.parameters(), lr=LR)
memory = deque(maxlen=MEMORY_SIZE)

def select_action(state):
    global EPSILON
    if random.random() < EPSILON:
        return env.action_space.sample()
    else:
        state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        with torch.no_grad():
            return policy_net(state).argmax().item()

def train():
    if len(memory) < BATCH_SIZE:
        return

    batch = random.sample(memory, BATCH_SIZE)
    states, actions, rewards, next_states, dones = zip(*batch)

    states = torch.tensor(states, dtype=torch.float32)
    actions = torch.tensor(actions, dtype=torch.int64).unsqueeze(1)
    rewards = torch.tensor(rewards, dtype=torch.float32).unsqueeze(1)
    next_states = torch.tensor(next_states, dtype=torch.float32)
    dones = torch.tensor(dones, dtype=torch.float32).unsqueeze(1)

    q_values = policy_net(states).gather(1, actions)

    with torch.no_grad():
        next_q_values = target_net(next_states).max(1, keepdim=True)[0]
        target_values = rewards + (1 - dones) * GAMMA * next_q_values

    loss = F.mse_loss(q_values, target_values)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Main training loop
num_episodes = 500
for episode in range(num_episodes):
    state = env.reset()  # Just get the state directly
    total_reward = 0
    done = False

    while not done:
        action = select_action(state)
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        memory.append((state, action, reward, next_state, done))
        state = next_state
        total_reward += reward

        train()

    if EPSILON > EPSILON_MIN:
        EPSILON *= EPSILON_DECAY

    if episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

    print(f"Episode {episode + 1}, Total Reward: {total_reward}")

env.close()

Episode 1, Total Reward: 17.0
Episode 2, Total Reward: 10.0


<ipython-input-4-2388e344436c>:61: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  states = torch.tensor(states, dtype=torch.float32)


Episode 3, Total Reward: 41.0
Episode 4, Total Reward: 42.0
Episode 5, Total Reward: 17.0
Episode 6, Total Reward: 13.0
Episode 7, Total Reward: 12.0
Episode 8, Total Reward: 9.0
Episode 9, Total Reward: 29.0
Episode 10, Total Reward: 14.0
Episode 11, Total Reward: 22.0
Episode 12, Total Reward: 16.0
Episode 13, Total Reward: 15.0
Episode 14, Total Reward: 44.0
Episode 15, Total Reward: 16.0
Episode 16, Total Reward: 17.0
Episode 17, Total Reward: 24.0
Episode 18, Total Reward: 15.0
Episode 19, Total Reward: 15.0
Episode 20, Total Reward: 28.0
Episode 21, Total Reward: 20.0
Episode 22, Total Reward: 30.0
Episode 23, Total Reward: 60.0
Episode 24, Total Reward: 49.0
Episode 25, Total Reward: 13.0
Episode 26, Total Reward: 13.0
Episode 27, Total Reward: 11.0
Episode 28, Total Reward: 18.0
Episode 29, Total Reward: 15.0
Episode 30, Total Reward: 13.0
Episode 31, Total Reward: 24.0
Episode 32, Total Reward: 21.0
Episode 33, Total Reward: 24.0
Episode 34, Total Reward: 11.0
Episode 35, Tota

In [5]:
import gym
import numpy as np

# Create CartPole environment with new step API
env = gym.make('CartPole-v1', render_mode="human", new_step_api=True)

num_episodes = 10
for episode in range(num_episodes):
    state = env.reset()[0]  # Get initial state
    terminated = False
    truncated = False
    total_reward = 0

    while not (terminated or truncated):
        env.render()  # Show environment
        action = env.action_space.sample()  # Select random action
        observation, reward, terminated, truncated, info = env.step(action)
        total_reward += reward

    print(f"Episode {episode + 1}: Total Reward = {total_reward}")

env.close()

Episode 1: Total Reward = 12.0
Episode 2: Total Reward = 35.0
Episode 3: Total Reward = 10.0
Episode 4: Total Reward = 15.0
Episode 5: Total Reward = 41.0
Episode 6: Total Reward = 68.0
Episode 7: Total Reward = 26.0
Episode 8: Total Reward = 29.0
Episode 9: Total Reward = 28.0
Episode 10: Total Reward = 16.0
